In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# Read the CSV file
df = pd.read_csv("sale_prop_data.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2403 entries, 0 to 2402
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                2403 non-null   int64  
 1   address                   2403 non-null   object 
 2   price                     2403 non-null   object 
 3   beds                      2403 non-null   object 
 4   baths                     2403 non-null   object 
 5   area                      2403 non-null   object 
 6   city                      2403 non-null   object 
 7   zipcode                   2403 non-null   int64  
 8   Time on Redfin            2065 non-null   object 
 9   Property Type             2403 non-null   object 
 10  Year Built                1871 non-null   float64
 11  Community                 2327 non-null   object 
 12  Lot Size                  1759 non-null   object 
 13  List Price                2363 non-null   object 
 14  Est. Mo.

In [3]:
df.head(10)

,Unnamed: 0,address,price,beds,baths,area,city,zipcode,Time on Redfin,Property Type,...,Community,Lot Size,List Price,Est. Mo. Payment,Redfin Estimate,Price/Sq.Ft.,Buyer’s Agent Commission,HOA Dues,Style,Baths
0,0,"41247 Apricot Ln,Fremont, CA 94539","$1,999,888",4,2,"1,698 Sq ft",Fremont,94539,5 days,Single Family Residential,...,FREMONT,"6,720 Sq. Ft.","$1,999,888","$14,526","$2,134,398","$1,178",2.5%,NaN,NaN,NaN
1,1,"34116 Pavia Ter,Fremont, CA 94555","$1,180,000",3,3.5,"1,387 Sq ft",Fremont,94555,1 day,Condo,...,ARDENWOOD AREA,0.91 Acres,"$1,180,000","$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary,NaN
2,2,"36778 Oak St,Fremont, CA 94536","$1,499,888",3,2,"1,610 Sq ft",Fremont,94536,1 day,Single Family Residential,...,FAIR OAKS,"6,978 Sq. Ft.","$1,499,888","$10,894","$1,513,958",$932,2.5%,NaN,Ranch,NaN
3,3,"4046 Abbey Ter #113,Fremont, CA 94536","$598,500",2,2,823 Sq ft,Fremont,94536,1 day,Condo,...,CENTRAL PARK,0.58 Acres,"$598,500","$4,757","$598,791",$727,2.5%,$410/month,Contemporary,NaN
4,4,"345 Torrano Cmn,Fremont, CA 94536","$799,888",2,1,"1,016 Sq ft",Fremont,94536,1 day,Townhouse,...,NILES THE TREES,"2,448 Sq. Ft.","$799,888","$6,150","$798,177",$787,2.5%,$340/month,NaN,NaN
5,5,"3374 Wolcott Cmn,FREMONT, CA 94538","$785,000",2,2,"1,280 Sq ft",Fremont,94538,1 day,Condo,...,FREMONT,NaN,"$785,000","$6,123","$784,377",$613,2.5%,$421,NaN,NaN
6,6,"37955 Camden St,FREMONT, CA 94536","$1,100,000",3,2,"1,530 Sq ft",Fremont,94536,1 day,Single Family Home,...,FREMONT,"6,237 Sq. Ft.","$1,100,000","$8,023","$1,108,121",$719,2.5%,$33,NaN,NaN
7,7,"3218 Winding Vista Cmn,Fremont, CA 94539","$3,550,000",4,3.5,"3,442 Sq ft",Fremont,94539,1 day,Single Family Residential,...,PONDEROSA HTS,0.55 Acres,"$3,550,000","$26,000","$3,553,718","$1,031",2.5%,$215/month,Mediterranean,NaN
8,8,"4120 Woodcrest Dr,FREMONT, CA 94538","$1,399,000",3,2,"1,120 Sq ft",Fremont,94538,2 days,Single Family Home,...,FREMONT,"5,995 Sq. Ft.","$1,399,000","$10,162","$1,349,353","$1,249",2.5%,NaN,NaN,NaN
9,9,"5533 Azalea Way,NEWARK, CA 94560","$1,279,980",4,3.5,"2,026 Sq ft",Fremont,94560,2 days,Condo (Co-op),...,NEWARK,NaN,"$1,279,980","$8,276","$1,269,155",$632,2.5%,$312,NaN,NaN


In [4]:
# Drop duplicate column
df = df.drop(columns=['Baths', 'List Price', 'Unnamed: 0'])
df

,address,price,beds,baths,area,city,zipcode,Time on Redfin,Property Type,Year Built,Community,Lot Size,Est. Mo. Payment,Redfin Estimate,Price/Sq.Ft.,Buyer’s Agent Commission,HOA Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539","$1,999,888",4,2,"1,698 Sq ft",Fremont,94539,5 days,Single Family Residential,1960.0,FREMONT,"6,720 Sq. Ft.","$14,526","$2,134,398","$1,178",2.5%,NaN,NaN
1,"34116 Pavia Ter,Fremont, CA 94555","$1,180,000",3,3.5,"1,387 Sq ft",Fremont,94555,1 day,Condo,2012.0,ARDENWOOD AREA,0.91 Acres,"$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536","$1,499,888",3,2,"1,610 Sq ft",Fremont,94536,1 day,Single Family Residential,1961.0,FAIR OAKS,"6,978 Sq. Ft.","$10,894","$1,513,958",$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536","$598,500",2,2,823 Sq ft,Fremont,94536,1 day,Condo,1986.0,CENTRAL PARK,0.58 Acres,"$4,757","$598,791",$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536","$799,888",2,1,"1,016 Sq ft",Fremont,94536,1 day,Townhouse,1975.0,NILES THE TREES,"2,448 Sq. Ft.","$6,150","$798,177",$787,2.5%,$340/month,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,"151 Wellington Ave,Daly City, CA 94014","$1,150,000",2,1,980 Sq ft,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,597 Sq. Ft.","$8,353","$1,082,742","$1,173",NaN,NaN,"Traditional, Other"
2399,"127 Wellington Ave,Daly City, CA 94014","$850,000",2,1,"1,270 Sq ft",Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,094 Sq. Ft.","$6,174","$829,959",$669,NaN,NaN,"Traditional, Other"
2400,"475 Irvington St,Daly City, CA 94014","$875,000",2,1,980 Sq ft,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"1,850 Sq. Ft.","$6,356","$842,589",$893,NaN,NaN,Traditional
2401,"68 Lausanne Ave,DALY CITY, CA 94014","$1,250,000",5,4,"4,348 Sq ft",Daly City,94014,375 days,Triplex,1965.0,DALY CITY,"2,500 Sq. Ft.","$9,079","$1,189,034",$287,2.5%,NaN,NaN


In [5]:
# Rename columns
df.rename(columns={'address':'Address','price':'Price',
                      'beds':'Beds','baths':'Baths',
                      'area':'Area_Sqft','city':'City',
                      'zipcode':'Zipcode',
                      'Time on Redfin':'Time_on_Redfin(days)','Year Built':'Year_built',
                      'Community':'Community','Lot Size':'Lot_Size_Sqft','Est. Mo. Payment':'Est_Monthly_Payt',
                      'Redfin Estimate':'Est_Redfin_Price','Price/Sq.Ft':'Price/Sq.Ft','Buyer’s Agent Commission':'Buyer_Agent_Commission',
                      'HOA Dues':'HOA_Dues','Style':'Style'}, inplace=True)
df


,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539","$1,999,888",4,2,"1,698 Sq ft",Fremont,94539,5 days,Single Family Residential,1960.0,FREMONT,"6,720 Sq. Ft.","$14,526","$2,134,398","$1,178",2.5%,NaN,NaN
1,"34116 Pavia Ter,Fremont, CA 94555","$1,180,000",3,3.5,"1,387 Sq ft",Fremont,94555,1 day,Condo,2012.0,ARDENWOOD AREA,0.91 Acres,"$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536","$1,499,888",3,2,"1,610 Sq ft",Fremont,94536,1 day,Single Family Residential,1961.0,FAIR OAKS,"6,978 Sq. Ft.","$10,894","$1,513,958",$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536","$598,500",2,2,823 Sq ft,Fremont,94536,1 day,Condo,1986.0,CENTRAL PARK,0.58 Acres,"$4,757","$598,791",$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536","$799,888",2,1,"1,016 Sq ft",Fremont,94536,1 day,Townhouse,1975.0,NILES THE TREES,"2,448 Sq. Ft.","$6,150","$798,177",$787,2.5%,$340/month,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,"151 Wellington Ave,Daly City, CA 94014","$1,150,000",2,1,980 Sq ft,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,597 Sq. Ft.","$8,353","$1,082,742","$1,173",NaN,NaN,"Traditional, Other"
2399,"127 Wellington Ave,Daly City, CA 94014","$850,000",2,1,"1,270 Sq ft",Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,094 Sq. Ft.","$6,174","$829,959",$669,NaN,NaN,"Traditional, Other"
2400,"475 Irvington St,Daly City, CA 94014","$875,000",2,1,980 Sq ft,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"1,850 Sq. Ft.","$6,356","$842,589",$893,NaN,NaN,Traditional
2401,"68 Lausanne Ave,DALY CITY, CA 94014","$1,250,000",5,4,"4,348 Sq ft",Daly City,94014,375 days,Triplex,1965.0,DALY CITY,"2,500 Sq. Ft.","$9,079","$1,189,034",$287,2.5%,NaN,NaN


In [6]:
# Check duplicate rows
duplicate_rows = df[df.duplicated()]
duplicate_rows


,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
535,"Plan 1,San Jose, CA 95120","$2,608,000+",3,3.5,"3,669 Sq ft",San Jose,95120,NaN,Single Family Residential,NaN,Almaden Heights,NaN,"$18,943",NaN,$711,NaN,NaN,NaN
536,"Plan 3,San Jose, CA 95120","$2,833,000+",4,3.5,"3,603 Sq ft",San Jose,95120,NaN,Single Family Residential,NaN,Almaden Heights,NaN,"$20,577",NaN,$786,NaN,NaN,NaN
537,"Zinnia,Milpitas, CA 95035","$1,461,995+",3,3,"1,948 Sq ft",San Jose,95035,NaN,Townhouse,NaN,Parkside at Tarob Court,NaN,"$10,619",NaN,$751,NaN,NaN,NaN
538,"Periwinkle,Milpitas, CA 95035","$1,461,995+",3,2.5,"1,948 Sq ft",San Jose,95035,NaN,Townhouse,NaN,Parkside at Tarob Court,NaN,"$10,619",NaN,$751,NaN,NaN,NaN
539,"Violet,Milpitas, CA 95035","$1,181,995+",3,3,"1,475 Sq ft",San Jose,95035,NaN,Townhouse,NaN,Parkside at Tarob Court,NaN,"$8,585",NaN,$801,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,"Plan 6,San Jose, CA 95119","$1,150,900+",3,3.5,"1,774 Sq ft",San Jose,95119,NaN,Condo,NaN,Lotus at UrbanOak,NaN,"$8,359",NaN,$649,NaN,NaN,"CondoOnly,MasterPlanned"
771,"Plan 2,San Jose, CA 95119","$994,900+",2,2.5,"1,355 Sq ft",San Jose,95119,NaN,Condo,NaN,Lotus at UrbanOak,NaN,"$7,226",NaN,$734,NaN,NaN,"CondoOnly,MasterPlanned"
772,"The Welford,San Jose, CA 95124","$3,260,000+",5,5.5,"3,400 Sq ft",San Jose,95124,NaN,Single Family Residential,NaN,Rosewood at Camden Park,NaN,"$23,679",NaN,$959,NaN,NaN,NaN
773,"The Ascot with 1 Br ADU,San Jose, CA 95124","$3,195,000+",4,4.5,"3,208 Sq ft",San Jose,95124,NaN,Single Family Residential,NaN,Rosewood at Camden Park,NaN,"$23,207",NaN,$996,NaN,NaN,NaN


In [7]:
# Drop duplicate rows
df = df.drop_duplicates()
df

,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539","$1,999,888",4,2,"1,698 Sq ft",Fremont,94539,5 days,Single Family Residential,1960.0,FREMONT,"6,720 Sq. Ft.","$14,526","$2,134,398","$1,178",2.5%,NaN,NaN
1,"34116 Pavia Ter,Fremont, CA 94555","$1,180,000",3,3.5,"1,387 Sq ft",Fremont,94555,1 day,Condo,2012.0,ARDENWOOD AREA,0.91 Acres,"$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536","$1,499,888",3,2,"1,610 Sq ft",Fremont,94536,1 day,Single Family Residential,1961.0,FAIR OAKS,"6,978 Sq. Ft.","$10,894","$1,513,958",$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536","$598,500",2,2,823 Sq ft,Fremont,94536,1 day,Condo,1986.0,CENTRAL PARK,0.58 Acres,"$4,757","$598,791",$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536","$799,888",2,1,"1,016 Sq ft",Fremont,94536,1 day,Townhouse,1975.0,NILES THE TREES,"2,448 Sq. Ft.","$6,150","$798,177",$787,2.5%,$340/month,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,"151 Wellington Ave,Daly City, CA 94014","$1,150,000",2,1,980 Sq ft,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,597 Sq. Ft.","$8,353","$1,082,742","$1,173",NaN,NaN,"Traditional, Other"
2399,"127 Wellington Ave,Daly City, CA 94014","$850,000",2,1,"1,270 Sq ft",Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,094 Sq. Ft.","$6,174","$829,959",$669,NaN,NaN,"Traditional, Other"
2400,"475 Irvington St,Daly City, CA 94014","$875,000",2,1,980 Sq ft,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"1,850 Sq. Ft.","$6,356","$842,589",$893,NaN,NaN,Traditional
2401,"68 Lausanne Ave,DALY CITY, CA 94014","$1,250,000",5,4,"4,348 Sq ft",Daly City,94014,375 days,Triplex,1965.0,DALY CITY,"2,500 Sq. Ft.","$9,079","$1,189,034",$287,2.5%,NaN,NaN


In [8]:
# Dealing with missing values

In [9]:
# Check for '—' in the DataFrame
contains_dash = df.apply(lambda x: x.astype(str).str.contains('—')).any()

# Print columns that contain '—'
columns_with_dash = contains_dash[contains_dash == True].index.tolist()
print("Columns containing '—':", columns_with_dash)

Columns containing '—': ['Price', 'Beds', 'Baths', 'Area_Sqft']


In [10]:
# Count occurrences of '—' in the specified column for each row
dash_row_counts = df[df['Price'].astype(str).str.contains('—')].shape[0]
print("Number of rows with '—' in the", 'Price', "column:", dash_row_counts)

dash_row_counts = df[df['Beds'].astype(str).str.contains('—')].shape[0]
print("Number of rows with '—' in the", 'Beds', "column:", dash_row_counts)

dash_row_counts = df[df['Baths'].astype(str).str.contains('—')].shape[0]
print("Number of rows with '—' in the", 'Baths', "column:", dash_row_counts)

dash_row_counts = df[df['Area_Sqft'].astype(str).str.contains('—')].shape[0]
print("Number of rows with '—' in the", 'Area_Sqft', "column:", dash_row_counts)


Number of rows with '—' in the Price column: 1
Number of rows with '—' in the Beds column: 218
Number of rows with '—' in the Baths column: 255
Number of rows with '—' in the Area_Sqft column: 32


In [11]:
# Replace '—' with 0
# df['Price'] = df['Price'].str.replace('—', '0')
# df['Beds'] = df['Beds'].str.replace('—', '0')
# df['Baths'] = df['Baths'].str.replace('—', '0')
# df['Area_Sqft'] = df['Area_Sqft'].str.replace('—', '0')

In [12]:
def check_range(value):
    if '-' in value:
        return value.split('-')[1]
    
    return value


df['Beds'] = df['Beds'].apply(check_range)
df['Baths'] = df['Baths'].apply(check_range)


C:\Users\STSC\AppData\Local\Temp\ipykernel_25344\1065328987.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Beds'] = df['Beds'].apply(check_range)
C:\Users\STSC\AppData\Local\Temp\ipykernel_25344\1065328987.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Baths'] = df['Baths'].apply(check_range)


In [13]:
def extract_number(value):
    # Use regular expression to find numeric values
#     print(value)
    numeric_values = ''.join(re.findall(r'\d+', value))
    if numeric_values:
        return int(''.join(numeric_values))
    else:
        return None  

df['Price'] = df['Price'].apply(extract_number)
df

C:\Users\STSC\AppData\Local\Temp\ipykernel_25344\2758520514.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'] = df['Price'].apply(extract_number)


,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539",1999888.0,4,2,"1,698 Sq ft",Fremont,94539,5 days,Single Family Residential,1960.0,FREMONT,"6,720 Sq. Ft.","$14,526","$2,134,398","$1,178",2.5%,NaN,NaN
1,"34116 Pavia Ter,Fremont, CA 94555",1180000.0,3,3.5,"1,387 Sq ft",Fremont,94555,1 day,Condo,2012.0,ARDENWOOD AREA,0.91 Acres,"$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536",1499888.0,3,2,"1,610 Sq ft",Fremont,94536,1 day,Single Family Residential,1961.0,FAIR OAKS,"6,978 Sq. Ft.","$10,894","$1,513,958",$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536",598500.0,2,2,823 Sq ft,Fremont,94536,1 day,Condo,1986.0,CENTRAL PARK,0.58 Acres,"$4,757","$598,791",$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536",799888.0,2,1,"1,016 Sq ft",Fremont,94536,1 day,Townhouse,1975.0,NILES THE TREES,"2,448 Sq. Ft.","$6,150","$798,177",$787,2.5%,$340/month,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,"151 Wellington Ave,Daly City, CA 94014",1150000.0,2,1,980 Sq ft,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,597 Sq. Ft.","$8,353","$1,082,742","$1,173",NaN,NaN,"Traditional, Other"
2399,"127 Wellington Ave,Daly City, CA 94014",850000.0,2,1,"1,270 Sq ft",Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,094 Sq. Ft.","$6,174","$829,959",$669,NaN,NaN,"Traditional, Other"
2400,"475 Irvington St,Daly City, CA 94014",875000.0,2,1,980 Sq ft,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"1,850 Sq. Ft.","$6,356","$842,589",$893,NaN,NaN,Traditional
2401,"68 Lausanne Ave,DALY CITY, CA 94014",1250000.0,5,4,"4,348 Sq ft",Daly City,94014,375 days,Triplex,1965.0,DALY CITY,"2,500 Sq. Ft.","$9,079","$1,189,034",$287,2.5%,NaN,NaN


In [14]:
def clean_and_convert_area(area):
    area= str(area)
    if pd.notna(area) and area.strip() != "" and area.strip()!='<NA>':
        area = area.replace('Sq ft', '').strip()
        cleaned_values = area.replace(',', '').split('-')
        try:
            if len(cleaned_values) == 2:
                avg_area = (int(cleaned_values[0]) + int(cleaned_values[1])) / 2
                return avg_area
            elif len(cleaned_values) == 1:
                return int(cleaned_values[0])
        except:
            return area
        return None

df['Area_Sqft'] = df['Area_Sqft'].apply(clean_and_convert_area)

C:\Users\STSC\AppData\Local\Temp\ipykernel_25344\3949523148.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Area_Sqft'] = df['Area_Sqft'].apply(clean_and_convert_area)


In [15]:
# Missing values in df
df.isnull().sum()

Address                      0
Price                        1
Beds                         0
Baths                        0
Area_Sqft                    0
City                         0
Zipcode                      0
Time_on_Redfin(days)       106
Property Type                0
Year_built                 300
Community                   68
Lot_Size_Sqft              405
Est_Monthly_Payt             7
Est_Redfin_Price           350
Price/Sq.Ft.               105
Buyer_Agent_Commission     606
HOA_Dues                  1467
Style                     1258
dtype: int64

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2164 entries, 0 to 2402
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Address                 2164 non-null   object 
 1   Price                   2163 non-null   float64
 2   Beds                    2164 non-null   object 
 3   Baths                   2164 non-null   object 
 4   Area_Sqft               2164 non-null   object 
 5   City                    2164 non-null   object 
 6   Zipcode                 2164 non-null   int64  
 7   Time_on_Redfin(days)    2058 non-null   object 
 8   Property Type           2164 non-null   object 
 9   Year_built              1864 non-null   float64
 10  Community               2096 non-null   object 
 11  Lot_Size_Sqft           1759 non-null   object 
 12  Est_Monthly_Payt        2157 non-null   object 
 13  Est_Redfin_Price        1814 non-null   object 
 14  Price/Sq.Ft.            2059 non-null   

In [17]:
# Check if Area_Sqft equal to 0
zero_area_rows = df[df['Area_Sqft'] == '0 Sq ft']
zero_area_rows


,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style


In [18]:
# Remove rows with Area_Sqft equal to 0
df = df[df['Area_Sqft'] != '0 Sq ft']

# Reset the index if needed
df = df.reset_index(drop=True)
df

,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539",1999888.0,4,2,1698,Fremont,94539,5 days,Single Family Residential,1960.0,FREMONT,"6,720 Sq. Ft.","$14,526","$2,134,398","$1,178",2.5%,NaN,NaN
1,"34116 Pavia Ter,Fremont, CA 94555",1180000.0,3,3.5,1387,Fremont,94555,1 day,Condo,2012.0,ARDENWOOD AREA,0.91 Acres,"$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536",1499888.0,3,2,1610,Fremont,94536,1 day,Single Family Residential,1961.0,FAIR OAKS,"6,978 Sq. Ft.","$10,894","$1,513,958",$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536",598500.0,2,2,823,Fremont,94536,1 day,Condo,1986.0,CENTRAL PARK,0.58 Acres,"$4,757","$598,791",$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536",799888.0,2,1,1016,Fremont,94536,1 day,Townhouse,1975.0,NILES THE TREES,"2,448 Sq. Ft.","$6,150","$798,177",$787,2.5%,$340/month,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,"151 Wellington Ave,Daly City, CA 94014",1150000.0,2,1,980,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,597 Sq. Ft.","$8,353","$1,082,742","$1,173",NaN,NaN,"Traditional, Other"
2160,"127 Wellington Ave,Daly City, CA 94014",850000.0,2,1,1270,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"3,094 Sq. Ft.","$6,174","$829,959",$669,NaN,NaN,"Traditional, Other"
2161,"475 Irvington St,Daly City, CA 94014",875000.0,2,1,980,Daly City,94014,332 days,Single Family Residential,1912.0,Daly City,"1,850 Sq. Ft.","$6,356","$842,589",$893,NaN,NaN,Traditional
2162,"68 Lausanne Ave,DALY CITY, CA 94014",1250000.0,5,4,4348,Daly City,94014,375 days,Triplex,1965.0,DALY CITY,"2,500 Sq. Ft.","$9,079","$1,189,034",$287,2.5%,NaN,NaN


In [19]:
# Check for rows with both "Beds" and "Baths" equal to 0
zero_beds_baths = df[(df['Beds'] == '0') & (df['Baths'] == '0')]
zero_beds_baths

,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style


In [20]:
#If both beds and Bath have 0, it shows the property is empty. Remove such rows
df = df.drop(zero_beds_baths.index)

# Reset the index, if needed
df.reset_index(drop=True, inplace=True)

In [21]:
# Remove the measurement units of "Area_Sqft"
# df['Area_Sqft'] = pd.to_numeric(df['Area_Sqft'].str.replace(' Sq ft', '', regex=False).str.replace(',', ''), errors='coerce').astype(float).astype('Int64', errors='ignore')

# Remove the measurement units of "Price"
# df['Price'] = pd.to_numeric(df['Price'].str.replace('[\$,]', '', regex=True), errors='coerce').astype(float).astype('Int64', errors='ignore')

# Remove the measurement units of "Time_on_Redfin(days)"
df['Time_on_Redfin(days)'] = pd.to_numeric(df['Time_on_Redfin(days)'].str.replace(' days?', '', regex=True), errors='coerce').astype(float).astype('Int64', errors='ignore')

df

,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539",1999888.0,4,2,1698,Fremont,94539,5,Single Family Residential,1960.0,FREMONT,"6,720 Sq. Ft.","$14,526","$2,134,398","$1,178",2.5%,NaN,NaN
1,"34116 Pavia Ter,Fremont, CA 94555",1180000.0,3,3.5,1387,Fremont,94555,1,Condo,2012.0,ARDENWOOD AREA,0.91 Acres,"$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536",1499888.0,3,2,1610,Fremont,94536,1,Single Family Residential,1961.0,FAIR OAKS,"6,978 Sq. Ft.","$10,894","$1,513,958",$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536",598500.0,2,2,823,Fremont,94536,1,Condo,1986.0,CENTRAL PARK,0.58 Acres,"$4,757","$598,791",$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536",799888.0,2,1,1016,Fremont,94536,1,Townhouse,1975.0,NILES THE TREES,"2,448 Sq. Ft.","$6,150","$798,177",$787,2.5%,$340/month,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,"151 Wellington Ave,Daly City, CA 94014",1150000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,"3,597 Sq. Ft.","$8,353","$1,082,742","$1,173",NaN,NaN,"Traditional, Other"
2160,"127 Wellington Ave,Daly City, CA 94014",850000.0,2,1,1270,Daly City,94014,332,Single Family Residential,1912.0,Daly City,"3,094 Sq. Ft.","$6,174","$829,959",$669,NaN,NaN,"Traditional, Other"
2161,"475 Irvington St,Daly City, CA 94014",875000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,"1,850 Sq. Ft.","$6,356","$842,589",$893,NaN,NaN,Traditional
2162,"68 Lausanne Ave,DALY CITY, CA 94014",1250000.0,5,4,4348,Daly City,94014,375,Triplex,1965.0,DALY CITY,"2,500 Sq. Ft.","$9,079","$1,189,034",$287,2.5%,NaN,NaN


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Address                 2164 non-null   object 
 1   Price                   2163 non-null   float64
 2   Beds                    2164 non-null   object 
 3   Baths                   2164 non-null   object 
 4   Area_Sqft               2164 non-null   object 
 5   City                    2164 non-null   object 
 6   Zipcode                 2164 non-null   int64  
 7   Time_on_Redfin(days)    2037 non-null   Int64  
 8   Property Type           2164 non-null   object 
 9   Year_built              1864 non-null   float64
 10  Community               2096 non-null   object 
 11  Lot_Size_Sqft           1759 non-null   object 
 12  Est_Monthly_Payt        2157 non-null   object 
 13  Est_Redfin_Price        1814 non-null   object 
 14  Price/Sq.Ft.            2059 non-null   

In [23]:
# Replace Time_on_Redfin(days) null values as 0 (assuming the entries for these properties are new)
df['Time_on_Redfin(days)'].fillna(0,inplace=True)
df

,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539",1999888.0,4,2,1698,Fremont,94539,5,Single Family Residential,1960.0,FREMONT,"6,720 Sq. Ft.","$14,526","$2,134,398","$1,178",2.5%,NaN,NaN
1,"34116 Pavia Ter,Fremont, CA 94555",1180000.0,3,3.5,1387,Fremont,94555,1,Condo,2012.0,ARDENWOOD AREA,0.91 Acres,"$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536",1499888.0,3,2,1610,Fremont,94536,1,Single Family Residential,1961.0,FAIR OAKS,"6,978 Sq. Ft.","$10,894","$1,513,958",$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536",598500.0,2,2,823,Fremont,94536,1,Condo,1986.0,CENTRAL PARK,0.58 Acres,"$4,757","$598,791",$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536",799888.0,2,1,1016,Fremont,94536,1,Townhouse,1975.0,NILES THE TREES,"2,448 Sq. Ft.","$6,150","$798,177",$787,2.5%,$340/month,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,"151 Wellington Ave,Daly City, CA 94014",1150000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,"3,597 Sq. Ft.","$8,353","$1,082,742","$1,173",NaN,NaN,"Traditional, Other"
2160,"127 Wellington Ave,Daly City, CA 94014",850000.0,2,1,1270,Daly City,94014,332,Single Family Residential,1912.0,Daly City,"3,094 Sq. Ft.","$6,174","$829,959",$669,NaN,NaN,"Traditional, Other"
2161,"475 Irvington St,Daly City, CA 94014",875000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,"1,850 Sq. Ft.","$6,356","$842,589",$893,NaN,NaN,Traditional
2162,"68 Lausanne Ave,DALY CITY, CA 94014",1250000.0,5,4,4348,Daly City,94014,375,Triplex,1965.0,DALY CITY,"2,500 Sq. Ft.","$9,079","$1,189,034",$287,2.5%,NaN,NaN


In [24]:
df.isnull().sum()

Address                      0
Price                        1
Beds                         0
Baths                        0
Area_Sqft                    0
City                         0
Zipcode                      0
Time_on_Redfin(days)         0
Property Type                0
Year_built                 300
Community                   68
Lot_Size_Sqft              405
Est_Monthly_Payt             7
Est_Redfin_Price           350
Price/Sq.Ft.               105
Buyer_Agent_Commission     606
HOA_Dues                  1467
Style                     1258
dtype: int64

In [25]:
# Treat NA Values in 'Price'
mean_price = df['Price'].mean().astype(int)

# Replace the NA values in the "Price" column with the mean value
df['Price'].fillna(mean_price, inplace=True)

In [26]:
# Treat NA Values in 'Price'
df['Year_built'].fillna('Unknown', inplace=True)

In [27]:
# Treat NA Values in 'Community'
df['Community'].fillna(df['City'], inplace=True)

In [28]:
# Treat NA Values in 'Price'
df['Style'].fillna('Unknown', inplace=True)
df

,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539",1999888.0,4,2,1698,Fremont,94539,5,Single Family Residential,1960.0,FREMONT,"6,720 Sq. Ft.","$14,526","$2,134,398","$1,178",2.5%,NaN,Unknown
1,"34116 Pavia Ter,Fremont, CA 94555",1180000.0,3,3.5,1387,Fremont,94555,1,Condo,2012.0,ARDENWOOD AREA,0.91 Acres,"$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536",1499888.0,3,2,1610,Fremont,94536,1,Single Family Residential,1961.0,FAIR OAKS,"6,978 Sq. Ft.","$10,894","$1,513,958",$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536",598500.0,2,2,823,Fremont,94536,1,Condo,1986.0,CENTRAL PARK,0.58 Acres,"$4,757","$598,791",$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536",799888.0,2,1,1016,Fremont,94536,1,Townhouse,1975.0,NILES THE TREES,"2,448 Sq. Ft.","$6,150","$798,177",$787,2.5%,$340/month,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,"151 Wellington Ave,Daly City, CA 94014",1150000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,"3,597 Sq. Ft.","$8,353","$1,082,742","$1,173",NaN,NaN,"Traditional, Other"
2160,"127 Wellington Ave,Daly City, CA 94014",850000.0,2,1,1270,Daly City,94014,332,Single Family Residential,1912.0,Daly City,"3,094 Sq. Ft.","$6,174","$829,959",$669,NaN,NaN,"Traditional, Other"
2161,"475 Irvington St,Daly City, CA 94014",875000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,"1,850 Sq. Ft.","$6,356","$842,589",$893,NaN,NaN,Traditional
2162,"68 Lausanne Ave,DALY CITY, CA 94014",1250000.0,5,4,4348,Daly City,94014,375,Triplex,1965.0,DALY CITY,"2,500 Sq. Ft.","$9,079","$1,189,034",$287,2.5%,NaN,Unknown


In [29]:
# Treat NA Values in 'Lot_Size_Sqft'

# Convert Acres to Sq. Ft.
def acres_to_sqft(value):
    if isinstance(value, float) and np.isnan(value):
        return np.nan
    elif isinstance(value, float):
        return int(value)
    elif 'Acres' in value:
        acres = float(value.split()[0].replace(',', ''))
        sqft = acres * 43560  # 1 Acre = 43,560 Sq. Ft.
        return int(sqft)
    elif 'Sq. Ft.' in value:
        sqft = float(value.split()[0].replace(',', ''))
        return int(sqft)
    else:
        return None

df['Lot_Size_Sqft'] = df['Lot_Size_Sqft'].apply(acres_to_sqft)
df

,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539",1999888.0,4,2,1698,Fremont,94539,5,Single Family Residential,1960.0,FREMONT,6720.0,"$14,526","$2,134,398","$1,178",2.5%,NaN,Unknown
1,"34116 Pavia Ter,Fremont, CA 94555",1180000.0,3,3.5,1387,Fremont,94555,1,Condo,2012.0,ARDENWOOD AREA,39639.0,"$8,855","$1,161,551",$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536",1499888.0,3,2,1610,Fremont,94536,1,Single Family Residential,1961.0,FAIR OAKS,6978.0,"$10,894","$1,513,958",$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536",598500.0,2,2,823,Fremont,94536,1,Condo,1986.0,CENTRAL PARK,25264.0,"$4,757","$598,791",$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536",799888.0,2,1,1016,Fremont,94536,1,Townhouse,1975.0,NILES THE TREES,2448.0,"$6,150","$798,177",$787,2.5%,$340/month,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,"151 Wellington Ave,Daly City, CA 94014",1150000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,3597.0,"$8,353","$1,082,742","$1,173",NaN,NaN,"Traditional, Other"
2160,"127 Wellington Ave,Daly City, CA 94014",850000.0,2,1,1270,Daly City,94014,332,Single Family Residential,1912.0,Daly City,3094.0,"$6,174","$829,959",$669,NaN,NaN,"Traditional, Other"
2161,"475 Irvington St,Daly City, CA 94014",875000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,1850.0,"$6,356","$842,589",$893,NaN,NaN,Traditional
2162,"68 Lausanne Ave,DALY CITY, CA 94014",1250000.0,5,4,4348,Daly City,94014,375,Triplex,1965.0,DALY CITY,2500.0,"$9,079","$1,189,034",$287,2.5%,NaN,Unknown


In [30]:
# Replace NaN values with the mean value in "Lot_Size_Sqft" column
mean_lot_size_sqft = df['Lot_Size_Sqft'].mean()
df['Lot_Size_Sqft'].fillna(mean_lot_size_sqft, inplace=True)


In [31]:
# Treat NA Values in 'Est_Redfin_Price' and 'Est_Monthly_Payt'
# Remove the measurement units of "Est_Redfin_Price"
df['Est_Redfin_Price'] = pd.to_numeric(df['Est_Redfin_Price'].str.replace('[\$,]', '', regex=True), errors='coerce').astype(float).astype('Int64', errors='ignore')

# Remove the measurement units of "Est_Monthly_Payt"
df['Est_Monthly_Payt'] = pd.to_numeric(df['Est_Monthly_Payt'].str.replace('[\$,]', '', regex=True), errors='coerce').astype(float).astype('Int64', errors='ignore')
df

,Address,Price,Beds,Baths,Area_Sqft,City,Zipcode,Time_on_Redfin(days),Property Type,Year_built,Community,Lot_Size_Sqft,Est_Monthly_Payt,Est_Redfin_Price,Price/Sq.Ft.,Buyer_Agent_Commission,HOA_Dues,Style
0,"41247 Apricot Ln,Fremont, CA 94539",1999888.0,4,2,1698,Fremont,94539,5,Single Family Residential,1960.0,FREMONT,6720.0,14526,2134398,"$1,178",2.5%,NaN,Unknown
1,"34116 Pavia Ter,Fremont, CA 94555",1180000.0,3,3.5,1387,Fremont,94555,1,Condo,2012.0,ARDENWOOD AREA,39639.0,8855,1161551,$851,2.5%,$284/month,Contemporary
2,"36778 Oak St,Fremont, CA 94536",1499888.0,3,2,1610,Fremont,94536,1,Single Family Residential,1961.0,FAIR OAKS,6978.0,10894,1513958,$932,2.5%,NaN,Ranch
3,"4046 Abbey Ter #113,Fremont, CA 94536",598500.0,2,2,823,Fremont,94536,1,Condo,1986.0,CENTRAL PARK,25264.0,4757,598791,$727,2.5%,$410/month,Contemporary
4,"345 Torrano Cmn,Fremont, CA 94536",799888.0,2,1,1016,Fremont,94536,1,Townhouse,1975.0,NILES THE TREES,2448.0,6150,798177,$787,2.5%,$340/month,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,"151 Wellington Ave,Daly City, CA 94014",1150000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,3597.0,8353,1082742,"$1,173",NaN,NaN,"Traditional, Other"
2160,"127 Wellington Ave,Daly City, CA 94014",850000.0,2,1,1270,Daly City,94014,332,Single Family Residential,1912.0,Daly City,3094.0,6174,829959,$669,NaN,NaN,"Traditional, Other"
2161,"475 Irvington St,Daly City, CA 94014",875000.0,2,1,980,Daly City,94014,332,Single Family Residential,1912.0,Daly City,1850.0,6356,842589,$893,NaN,NaN,Traditional
2162,"68 Lausanne Ave,DALY CITY, CA 94014",1250000.0,5,4,4348,Daly City,94014,375,Triplex,1965.0,DALY CITY,2500.0,9079,1189034,$287,2.5%,NaN,Unknown


In [32]:
# Replace NaN values with the mean value in "Est_Redfin_Price" column
mean_est_redfin_price = df['Est_Redfin_Price'].mean().astype(int)
df['Est_Redfin_Price'].fillna(mean_est_redfin_price, inplace=True)

# Replace NaN values with the mean value in "Est_Monthly_Payt" column
mean_est_monthly_price = df['Est_Monthly_Payt'].mean().astype(int)
df['Est_Monthly_Payt'].fillna(mean_est_monthly_price, inplace=True)

In [33]:
# Treat NA Values in 'HOA_Dues'
df['HOA_Dues'] = df['HOA_Dues'].str.replace('[,$/month]', '', regex=True)
df['HOA_Dues'].fillna(0,inplace=True)
df['HOA_Dues'] = df['HOA_Dues'].astype(float)

In [34]:
# Treat NA Values in 'Buyer_Agent_Commission'
df['Buyer_Agent_Commission'] = df['Buyer_Agent_Commission'].str.rstrip('%')

# Convert the column to numeric
df['Buyer_Agent_Commission'] = pd.to_numeric(df['Buyer_Agent_Commission'], errors='coerce')

# Calculate the mean commission excluding NaN values
mean_commission = df['Buyer_Agent_Commission'].mean()

# Replace null values with the mean
df['Buyer_Agent_Commission'].fillna(mean_commission, inplace=True)


In [35]:
# Recheck Null Values
df.isnull().sum()

Address                     0
Price                       0
Beds                        0
Baths                       0
Area_Sqft                   0
City                        0
Zipcode                     0
Time_on_Redfin(days)        0
Property Type               0
Year_built                  0
Community                   0
Lot_Size_Sqft               0
Est_Monthly_Payt            0
Est_Redfin_Price            0
Price/Sq.Ft.              105
Buyer_Agent_Commission      0
HOA_Dues                    0
Style                       0
dtype: int64

In [37]:
df.to_csv('sales_clean_data_new.csv')